# story dspy maker

## overall dag
1. input: accept story requests from users
2. prompt validation and rewriter: clean and validate prompts, remove harmful content
3. story generator: use DSPy to generate structured stories

In [15]:
import os
os.environ["GROQ_API_KEY"] = "sure"

In [2]:
import dspy
import re, os, json
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from enum import Enum

lm = dspy.LM(
    model="groq/llama-3.1-8b-instant",
    api_key=os.environ['GROQ_API_KEY'],
    max_tokens=2048,
    temperature=0.7
)

In [3]:
# testing groq config
lm("Say this is a test!", temperature=0.7)

["IT'S A TEST."]

## data structs

In [4]:
# dag input from user
@dataclass
class StoryRequest:
    """User's story request"""
    prompt: str
    age_group: str = "3-7"  # target age group
    story_length: str = "short"  # short, medium, long
    theme: Optional[str] = None  # adventure, friendship, learning, etc.

# dag part 2 validation and cleaning (using llm based for now)
@dataclass 
class ValidatedPrompt:
    """Cleaned and validated prompt"""
    original_prompt: str
    cleaned_prompt: str
    is_safe: bool
    removed_content: List[str]
    suggested_improvements: List[str]

# llm generation step
@dataclass
class Story:
    """Generated bedtime story"""
    title: str
    story: str
    characters: List[str]
    moral: str
    age_appropriate: bool
    estimated_reading_time: str


## input

In [5]:
class StoryRequestHandler:
    """Handles and normalizes user story requests"""
    
    def __init__(self):
        self.common_themes = [
            "adventure", "friendship", "learning", "animals", "magic", 
            "family", "kindness", "courage", "discovery", "helping others"
        ]
    
    def parse_request(self, user_input: str, age_group: str = "3-7", 
                     story_length: str = "short") -> StoryRequest:
        """Parse and normalize user input into a structured request"""

        # simple keyword classifier: extract potential themes from the prompt
        detected_theme = None
        for theme in self.common_themes:
            if theme.lower() in user_input.lower():
                detected_theme = theme
                break
        
        return StoryRequest(
            prompt=user_input.strip(),
            age_group=age_group,
            story_length=story_length,
            theme=detected_theme
        )

# testing
handler = StoryRequestHandler()
sample_requests = [
    "Tell me a story about a brave little mouse",
    "I want an adventure story with a dragon and friendship",
    "Story about learning to share toys"
]

for req in sample_requests:
    parsed = handler.parse_request(req)
    print(f"Request: {req}")
    print(f"Parsed: {parsed}")
    print()


Request: Tell me a story about a brave little mouse
Parsed: StoryRequest(prompt='Tell me a story about a brave little mouse', age_group='3-7', story_length='short', theme=None)

Request: I want an adventure story with a dragon and friendship
Parsed: StoryRequest(prompt='I want an adventure story with a dragon and friendship', age_group='3-7', story_length='short', theme='adventure')

Request: Story about learning to share toys
Parsed: StoryRequest(prompt='Story about learning to share toys', age_group='3-7', story_length='short', theme='learning')



## dag step 2


In [6]:
harmful_patterns = [
            r'\b(kill|death|die|hurt|pain|scary|frightening|nightmare)\b',
            r'\b(violence|fighting|war|gun|weapon|blood)\b',
            r'\b(hate|angry|sad|cry|mean|bad)\b',
        ]

In [7]:
class PromptValidator(dspy.Signature):
    """Validate and clean user prompts for child safety"""
    
    user_prompt = dspy.InputField(desc="Original user prompt for story request")
    age_group = dspy.InputField(desc="Target age group (e.g., 3-7, 5-10)")
    
    is_safe = dspy.OutputField(desc="Boolean: True if prompt is safe for children")
    cleaned_prompt = dspy.OutputField(desc="Cleaned version of the prompt, removing any inappropriate content")
    concerns = dspy.OutputField(desc="List of any safety concerns found (comma-separated)")
    suggestions = dspy.OutputField(desc="Suggestions to improve the prompt for better stories")

class PromptRewriter(dspy.Signature):
    """Rewrite prompts to be more suitable for children's stories"""
    
    cleaned_prompt = dspy.InputField(desc="Validated and cleaned prompt")
    age_group = dspy.InputField(desc="Target age group")
    theme = dspy.InputField(desc="Story theme if detected")
    
    enhanced_prompt = dspy.OutputField(desc="Enhanced prompt optimized for engaging children's stories")
    story_elements = dspy.OutputField(desc="Key story elements to include (characters, setting, conflict, resolution)")

class PromptPipeline:
    """Complete pipeline for prompt validation and enhancement"""
    
    def __init__(self):
        self.validator = dspy.ChainOfThought(PromptValidator)
        self.rewriter = dspy.ChainOfThought(PromptRewriter)

        # basic keyword checker
        self.harmful_patterns = [
            r'\b(kill|death|die|hurt|pain|scary|frightening|nightmare)\b',
            r'\b(violence|fighting|war|gun|weapon|blood)\b',
            r'\b(hate|angry|sad|cry|mean|bad)\b',
        ]
    
    def local_safety_check(self, prompt: str) -> tuple[bool, List[str]]:
        """Basic local safety check using regex patterns"""
        found_issues = []
        
        for pattern in self.harmful_patterns:
            matches = re.findall(pattern, prompt.lower())
            if matches:
                found_issues.extend(matches)
        
        is_safe = len(found_issues) == 0
        return is_safe, found_issues
    
    def validate_and_rewrite(self, request: StoryRequest) -> ValidatedPrompt:
        """Validate and rewrite prompt for safety and engagement"""
        
        # check against keywords
        local_safe, local_issues = self.local_safety_check(request.prompt)
        
        # retrieve pattern and clean prompt
        cleaned_prompt = request.prompt
        for pattern in self.harmful_patterns:
            cleaned_prompt = re.sub(pattern, '[child-friendly alternative]', cleaned_prompt, flags=re.IGNORECASE)
        
        # using LLM for deeper validation and enhancement
        validation_result = self.validator(
            user_prompt=cleaned_prompt,
            age_group=request.age_group
        )
        
        # if prompt is safe from the get-go (can remove validation step cuz its a waste redundant call but could leave it in if i dont find any matches)
        # TODO: better yet, could just combine it here
        if validation_result.is_safe and local_safe:
            rewrite_result = self.rewriter(
                cleaned_prompt=validation_result.cleaned_prompt,
                age_group=request.age_group,
                theme=request.theme or "general"
            )
            final_prompt = rewrite_result.enhanced_prompt
        else:
            final_prompt = validation_result.cleaned_prompt
        
        return ValidatedPrompt(
            original_prompt=request.prompt,
            cleaned_prompt=final_prompt,
            is_safe=validation_result.is_safe and local_safe,
            removed_content=local_issues + validation_result.concerns.split(', ') if validation_result.concerns else local_issues,
            suggested_improvements=validation_result.suggestions.split(', ') if validation_result.suggestions else []
        )

prompt_pipeline = PromptPipeline()
# self.harmful_patterns = [
#     r'\b(kill|death|die|hurt|pain|scary|frightening|nightmare)\b',
#     r'\b(violence|fighting|war|gun|weapon|blood)\b',
#     r'\b(hate|angry|sad|cry|mean|bad)\b',
# ]

test_prompts = [
    "Tell me a scary story about monsters",
    "A story about a brave little rabbit making friends",
    "Adventure with dragons that are mean and hurt people"
]

# prompt validation check with keywords from class (only doing the local one for now)
for prompt in test_prompts:
    request = StoryRequest(prompt=prompt)
    local_safe, issues = prompt_pipeline.local_safety_check(prompt)
    print(f"\nPrompt: {prompt}")
    print(f"Local safety check: {'Safe' if local_safe else 'Issues found'}")
    if issues:
        print(f"Issues: {issues}")


Prompt: Tell me a scary story about monsters
Local safety check: Issues found
Issues: ['scary']

Prompt: A story about a brave little rabbit making friends
Local safety check: Safe

Prompt: Adventure with dragons that are mean and hurt people
Local safety check: Issues found
Issues: ['hurt', 'mean']


## story maker


In [8]:
class StoryGenerator(dspy.Signature):
    """Generate engaging bedtime stories for children"""
    
    story_prompt = dspy.InputField(desc="Enhanced and validated story prompt")
    age_group = dspy.InputField(desc="Target age group (e.g., 3-7)")
    story_length = dspy.InputField(desc="Desired story length (short/medium/long)")
    theme = dspy.InputField(desc="Story theme")
    
    title = dspy.OutputField(desc="Engaging title for the story")
    story_text = dspy.OutputField(desc="Complete story text, age-appropriate and engaging")
    characters = dspy.OutputField(desc="Main characters in the story (comma-separated)")
    moral_lesson = dspy.OutputField(desc="Positive moral or lesson from the story")
    reading_time = dspy.OutputField(desc="Estimated reading time (e.g., '3-5 minutes')")

# could be absolutely redundant if client side parsing doesn't do it
class StoryStructureImprover(dspy.Signature):
    """Improve story structure for better TTS and voice cloning"""
    
    raw_story = dspy.InputField(desc="Generated story text")
    title = dspy.InputField(desc="Story title")
    
    improved_story = dspy.OutputField(desc="Story optimized for text-to-speech with clear pacing and pronunciation")
    dialogue_markers = dspy.OutputField(desc="Suggested voice/character indicators for voice cloning")
    pacing_notes = dspy.OutputField(desc="Notes for TTS pacing (pauses, emphasis, etc.)")

class StoryMaker:
    """Complete pipeline for generating bedtime stories"""
    
    def __init__(self):
        self.story_generator = dspy.ChainOfThought(StoryGenerator)
        self.structure_improver = dspy.ChainOfThought(StoryStructureImprover)
        
        # Length guidelines
        self.length_guidelines = {
            "short": "2-3 paragraphs, 150-300 words",
            "medium": "4-6 paragraphs, 300-600 words", 
            "long": "7-10 paragraphs, 600-1000 words"
        }
    
    def generate_story(self, validated_prompt: ValidatedPrompt, 
                      request: StoryRequest) -> Story:
        """Generate a complete bedtime story"""
        
        if not validated_prompt.is_safe:
            raise ValueError("Cannot generate story from unsafe prompt")
        
        # base
        story_result = self.story_generator(
            story_prompt=validated_prompt.cleaned_prompt,
            age_group=request.age_group,
            story_length=f"{request.story_length} ({self.length_guidelines.get(request.story_length, 'short')})",
            theme=request.theme or "general friendship and kindness"
        )
        
        # tts improvcement call
        improved_result = self.structure_improver(
            raw_story=story_result.story_text,
            title=story_result.title
        )
        
        return Story(
            title=story_result.title,
            story=improved_result.improved_story,
            characters=story_result.characters.split(', ') if story_result.characters else [],
            moral=story_result.moral_lesson,
            age_appropriate=True,  # Validated in previous step
            estimated_reading_time=story_result.reading_time
        )

    # again redundant and wasting tokens but let's see
    def generate_multiple_variants(self, validated_prompt: ValidatedPrompt, 
                                  request: StoryRequest, num_variants: int = 3) -> List[Story]:
        """Generate multiple story variants for A/B testing or user choice"""
        
        variants = []
        for i in range(num_variants):
            # updating prompt with variants
            variant_prompt = validated_prompt.cleaned_prompt + f" (variant {i+1}: focus on different aspects)"
            
            modified_validated = ValidatedPrompt(
                original_prompt=validated_prompt.original_prompt,
                cleaned_prompt=variant_prompt,
                is_safe=validated_prompt.is_safe,
                removed_content=validated_prompt.removed_content,
                suggested_improvements=validated_prompt.suggested_improvements
            )
            
            try:
                story = self.generate_story(modified_validated, request)
                variants.append(story)
            except Exception as e:
                print(f"Failed to generate variant {i+1}: {e}")
                continue
        
        return variants

story_pipeline = StoryMaker()
print("Story generation pipeline initialized!")
print("Length guidelines:", story_pipeline.length_guidelines)

Story generation pipeline initialized!
Length guidelines: {'short': '2-3 paragraphs, 150-300 words', 'medium': '4-6 paragraphs, 300-600 words', 'long': '7-10 paragraphs, 600-1000 words'}


## output


In [9]:
class BedtimeStoryFactory:
    """Complete end-to-end pipeline for bedtime story generation"""
    
    def __init__(self):
        self.request_handler = StoryRequestHandler()
        self.prompt_pipeline = PromptPipeline()
        self.story_pipeline = StoryMaker()
    
    def create_story(self, user_input: str, age_group: str = "3-7", 
                    story_length: str = "short", 
                    generate_variants: bool = False,
                    num_variants: int = 3) -> Dict[str, Any]:
        """Complete story creation pipeline"""
        
        pipeline_result = {
            'success': False,
            'request': None,
            'validated_prompt': None,
            'stories': [],
            'errors': []
        }
        
        try:
            request = self.request_handler.parse_request(user_input, age_group, story_length)
            pipeline_result['request'] = request
            validated_prompt = self.prompt_pipeline.validate_and_rewrite(request)
            pipeline_result['validated_prompt'] = validated_prompt
            
            if not validated_prompt.is_safe:
                pipeline_result['errors'].append("Prompt failed safety validation")
                return pipeline_result
            
            # generate story
            if generate_variants:
                stories = self.story_pipeline.generate_multiple_variants(
                    validated_prompt, request, num_variants
                )
            else:
                story = self.story_pipeline.generate_story(validated_prompt, request)
                stories = [story]
            
            pipeline_result['stories'] = stories
            
            if not stories:
                pipeline_result['errors'].append("Failed to generate any stories")
                return pipeline_result
            
            pipeline_result['success'] = True
            
        except Exception as e:
            pipeline_result['errors'].append(f"Pipeline error: {str(e)}")
        
        return pipeline_result
    
    def print_pipeline_results(self, result: Dict[str, Any]):
        if not result['success']:
            print("Pipeline failed!")
            for error in result['errors']:
                print(f"   Error: {error}")
            return
        
        # req
        if result['request']:
            req = result['request']
            print(f"req: {req.prompt}")
            print(f"age group: {req.age_group}")
            print(f"len: {req.story_length}")
            print(f"theme: {req.theme or 'General'}")
            print()
        
        # validation
        if result['validated_prompt']:
            val = result['validated_prompt']
            print(f"safety: {'Passed' if val.is_safe else 'Failed'}")
            if val.removed_content:
                print(f"removed: {val.removed_content}")
            print(f"enhanced: {val.cleaned_prompt[:100]}...")
            print()
        
        # result
        print(f"generated {len(result['stories'])} stories")
        print(result['stories'])

# Initialize the complete factory
story_factory = BedtimeStoryFactory()


## example


In [10]:
test_requests = [
    {
        'prompt': "A brave little rabbit who learns to share with forest friends",
        'age_group': "3-7",
        'length': "short"
    },
    {
        'prompt': "An adventure where a young dragon learns about friendship",
        'age_group': "5-10", 
        'length': "medium"
    },
    {
        'prompt': "A magical story about helping others and being kind",
        'age_group': "3-7",
        'length': "short"
    }
]

dspy.settings.configure(lm=lm)

results = []

for i, test_req in enumerate(test_requests, 1):
    print(f"TEST {i}: {test_req['prompt']}")
    
    result = story_factory.create_story(
        user_input=test_req['prompt'],
        age_group=test_req['age_group'],
        story_length=test_req['length'],
        generate_variants=False
    )
    results.append(result)
    
    story_factory.print_pipeline_results(result)
    print("\n\n")

TEST 1: A brave little rabbit who learns to share with forest friends
req: A brave little rabbit who learns to share with forest friends
age group: 3-7
len: short
theme: General

safety: Passed
removed: ['None']
enhanced: A little rabbit named Rosie lives in a cozy burrow in a forest filled with tall trees, busy beavers,...

generated 1 stories
[Story(title="Rosie's Big Storm Adventure: A Tale of Friendship and Sharing", story='Rosie the rabbit loved to play with her toys in her cozy burrow, surrounded by the warm glow of twinkling fireflies and the soothing sounds of gentle forest melodies. But whenever her friends, like Squeaky the squirrel and Benny the beaver, wanted to play with them too, Rosie would get frustrated. "Those are my toys!" she would exclaim, her voice rising with a hint of anxiety. One day, Squeaky approached Rosie and said, "Hey, friend! I want to play with your toys, but I promise I\'ll be gentle," his voice filled with a soothing calmness.\n\n(Rosie\'s voice) "Oka

In [11]:
results

[{'success': True,
  'request': StoryRequest(prompt='A brave little rabbit who learns to share with forest friends', age_group='3-7', story_length='short', theme=None),
  'validated_prompt': ValidatedPrompt(original_prompt='A brave little rabbit who learns to share with forest friends', cleaned_prompt="A little rabbit named Rosie lives in a cozy burrow in a forest filled with tall trees, busy beavers, and playful squirrels. Rosie loves to play with her toys, but she often gets frustrated when her friends want to play with them too. One day, she meets a friendly squirrel named Squeaky who teaches her the value of sharing and kindness. But when a big storm hits the forest, Rosie's toys are blown away, and she must learn to share and work together with her friends to retrieve them.", is_safe=True, removed_content=['None'], suggested_improvements=['The prompt is clean and suitable for the requested age group. However', 'to make it more engaging', 'you could consider specifying what the rab

In [12]:
import pprint as p

p.pprint(results[0]['stories'][0].title)
p.pprint(results[0]['stories'][0].story)

"Rosie's Big Storm Adventure: A Tale of Friendship and Sharing"
('Rosie the rabbit loved to play with her toys in her cozy burrow, surrounded '
 'by the warm glow of twinkling fireflies and the soothing sounds of gentle '
 'forest melodies. But whenever her friends, like Squeaky the squirrel and '
 'Benny the beaver, wanted to play with them too, Rosie would get frustrated. '
 '"Those are my toys!" she would exclaim, her voice rising with a hint of '
 'anxiety. One day, Squeaky approached Rosie and said, "Hey, friend! I want to '
 'play with your toys, but I promise I\'ll be gentle," his voice filled with a '
 'soothing calmness.\n'
 '\n'
 '(Rosie\'s voice) "Okay, you can play with them, but only if you share them '
 'with Benny too."\n'
 '\n'
 '(Squeaky\'s voice) "That\'s the spirit of friendship!" And so, Rosie, '
 'Squeaky, and Benny had a wonderful time playing together, their laughter and '
 'chirping filling the forest air.\n'
 '\n'
 'But then, a big storm hit the forest, its thu

In [13]:
p.pprint(results[1]['stories'][0].title)
p.pprint(results[1]['stories'][0].story)

'Ember and the Secret of Friendship'
('In a land far, far away, where the sun shone bright and the sky was painted '
 'with colors of blue and white, there lived a young dragon named Ember. Ember '
 'was a curious and adventurous dragon with a sparkle in her eye. She had '
 'scales that shimmered like the brightest gemstones, and her wings were as '
 'strong as the wind.\n'
 '\n'
 'One day, Ember decided to leave her cozy cave and explore the world beyond. '
 'As she soared through the skies, she spotted a hidden forest, surrounded by '
 'a babbling brook and towering trees.\n'
 '\n'
 'Ember (excitedly): Oh, wow! Look at that!\n'
 '\n'
 'A friendly rabbit named Rosie hopped out from behind a bush. Rosie had big, '
 'bright eyes and a fluffy tail that wagged with excitement.\n'
 '\n'
 'Rosie: (welcome tone) Hello there, young dragon! My name is Rosie. What '
 'brings you to our forest?\n'
 '\n'
 'Ember and Rosie quickly became fast friends. They spent their days exploring '
 'the hidden

In [14]:
p.pprint(results[2]['stories'][0].title)
p.pprint(results[2]['stories'][0].story)

"Rosie's Magical Smile"
('Once upon a time, in a lush meadow filled with vibrant flowers that swayed '
 'gently in the breeze, Rosie, a young rabbit with shimmering brown eyes that '
 'sparkled with curiosity, discovered a hidden cave behind a waterfall. As she '
 'explored the cave, she stumbled upon a sparkling amulet with an inscription '
 'that read: "Make someone smile, and magic will unfold." Rosie\'s eyes '
 'widened with wonder, and she decided to try it out.\n'
 '\n'
 '"Hey, Squeaky!" Rosie called out to her best friend, a lonely squirrel named '
 'Squeaky who was feeling down. "I found something amazing!"\n'
 '\n'
 'Squeaky looked up from his spot on the ground, his tail twitching with '
 'interest. "What is it, Rosie?"\n'
 '\n'
 'Rosie approached Squeaky, her paws wiggling with excitement. "It\'s this '
 'amulet," she said, holding it up. "It says that if I make someone smile, '
 'magic will unfold."\n'
 '\n'
 'Squeaky raised an eyebrow. "That sounds like a pretty big promis